In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [2]:
# Step 1: Load the Stanford Dogs dataset from TensorFlow Datasets
(train_data, test_data), info = tfds.load('stanford_dogs', split=['train', 'test'], 
                                          with_info=True, as_supervised=True)

2025-01-07 16:58:04.155981: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

...

Generating train examples...:   0%|          | 0/12000 [00:00<?, ? examples/s]

Shuffling /Users/kashishvarmaa/tensorflow_datasets/stanford_dogs/incomplete.6AUP18_0.2.0/stanford_dogs-train.t…

Generating test examples...:   0%|          | 0/8580 [00:00<?, ? examples/s]

Shuffling /Users/kashishvarmaa/tensorflow_datasets/stanford_dogs/incomplete.6AUP18_0.2.0/stanford_dogs-test.tf…

Dataset stanford_dogs downloaded and prepared to /Users/kashishvarmaa/tensorflow_datasets/stanford_dogs/0.2.0. Subsequent calls will reuse this data.


In [5]:
# Step 2: Preprocess the data (Resize and Normalize)
IMG_SIZE = (224, 224)

def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE)  # Resize images to 224x224
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1]
    return image, label

train_data = train_data.map(preprocess).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)
test_data = test_data.map(preprocess).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

In [6]:
# Step 3: Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [7]:
# Step 4: Add Custom Classification Layers 

# Add a global average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer
x = Dense(512, activation='relu')(x)

# Add a logistic layer for classification (Stanford Dogs dataset has 120 classes)
predictions = Dense(120, activation='softmax')(x)

# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

In [8]:
# Step 5: Freeze the Base Model Layers

for layer in base_model.layers:
    layer.trainable = False


In [9]:
# Step 6: Compile the Model

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
# Step 7: Train the Top Layers

# Add callbacks for better training control
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]

history = model.fit(train_data,
                    epochs=10,
                    validation_data=test_data,
                    callbacks=callbacks)

Epoch 1/10


2025-01-07 17:10:02.230656: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:376] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608


375/375 ━━━━━━━━━━━━━━━━━━━━ 3537s 9s/step - accuracy: 0.0100 - loss: 4.8531 - val_accuracy: 0.0108 - val_loss: 4.7722 - learning_rate: 0.0010
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 3618s 10s/step - accuracy: 0.0093 - loss: 4.7622 - val_accuracy: 0.0143 - val_loss: 4.7215 - learning_rate: 0.0010
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 3538s 9s/step - accuracy: 0.0152 - loss: 4.7156 - val_accuracy: 0.0191 - val_loss: 4.7019 - learning_rate: 0.0010
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 3702s 10s/step - accuracy: 0.0140 - loss: 4.6984 - val_accuracy: 0.0190 - val_loss: 4.6940 - learning_rate: 0.0010
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 4861s 13s/step - accuracy: 0.0169 - loss: 4.6892 - val_accuracy: 0.0190 - val_loss: 4.6871 - learning_rate: 0.0010
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 2185s 6s/step - accuracy: 0.0164 - loss: 4.6802 - val_accuracy: 0.0209 - val_loss: 4.6789 - learning_rate: 0.0010
Epoch 7/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 2415s 6s/step - accuracy: 0.0186 - loss: 4.6

## Adafilter Implementation

In [3]:
# Step 8: Fine-Tune the Important Filters (AdaFilter Implementation)

# 1. Identify Important Filters:

import numpy as np

def get_important_filters(model, layer_name, top_n=10):
    layer = model.get_layer(name=layer_name)
    weights, _ = layer.get_weights()
    importance_scores = np.mean(np.abs(weights), axis=(0, 1, 2))
    important_filters = np.argsort(importance_scores)[-top_n:]
    return important_filters

In [4]:
# 2. Fine-Tune Important Filters:

# Assuming 'conv4_block1_1_conv' is a layer name you want to fine-tune
important_filters = get_important_filters(model, 'conv4_block1_1_conv', top_n=10)

for i, layer in enumerate(base_model.layers):
    if 'conv' in layer.name:
        if i in important_filters:
            layer.trainable = True
        else:
            layer.trainable = False

# Compile the model with a lower learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
              

NameError: name 'model' is not defined

In [ ]:
# Step 9: Fine-Tune the Model

history_fine_tune = model.fit(train_data,
                              epochs=10,
                              validation_data=test_data,
                              callbacks=callbacks)

In [ ]:
# Step 10: Evaluate the Model

loss, accuracy = model.evaluate(test_data)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')